# Reinforcement Learning: 

In [ ]:
# This notebook documents the progress of my passion project at Metis.
# I'm including my learning notes, notes on setting up the environments, problem formulations, 
# as well as code samples for training an agent to play Atari games using Reinforcement Learning.

# This can be used as supplemental material for optional lectures in week 9 and 10 of the bootcamp
# Or a beginner's tutorial for final projects related to reinforcement learning.



# Lingqiang Kong
# 2017.08

## Objectives

In [10]:
* Understand Reinforcement Learning Problem setups and how it differs from supervised and unsupervised learning
* Understand the Agent-Environment interface
* Understand what MDPs (Markov Decision Processes) are and how to interpret transition diagrams
* Understand Value Functions, Action-Value Functions, and Policy Functions
* Understand the Bellman Equations and Bellman Optimality Equations for value functions and action-value functions

SyntaxError: invalid syntax (<ipython-input-10-77ba384a7a33>, line 2)

## What is Reinforcement Learning

In [2]:
Nature: Learning by interacting with environments
Learning what to do: How to map situations with actions, to get maximum reward
- Learner action affects later input
- Learner not told what action to take, must discover what actions yield the most reward by trying them out
- Action may not just affect immediate reward

## Differences between Reinforcement Learning and Supervised learning

In [4]:
Reinforcement learning is different from Supervise d Learning
- There is no supervisor, only a reward signal (RL goals are to maximize the reward or Expectation of reward)
- Feedback is delayed, not instantaenous
- Agent's action affects subsequent inputs

Supervised Learning
- There is a "correct" label for each training sample
- Time does not always matter (not always sequential)
- Future inputs are not affected

## Examples of Reinforcement Learning

In [ ]:
- Fly a stunt manuvour in a helicopter (that is very difficult for human to perform)
- Beat world champion in a Chess (Go?) game
- Play manu computer games better than human
- Make a humanoid robot walk

## Agent and environment

In [ ]:
At each time step t, the agent:
    - Execute Action A_t
    - Receives Oberservation O_t
    - Receives Reward R_t 

t increments after each step

We'll see this later in the OpenAI Gym Environment

## Markov States

A State S_t is considered **Markov** if and only if
$$ P[S_t+1 | S_t] = P[S_{t+1} | S_1, S_2, ..., S_t]$$

In [ ]:
* The future is independent of the past, given the present.
* Once the state is know, past history could be thrown away

## Markov Decision Process

# Open AI

OpenAI Gym is a toolkit for developing and comparing reinforcement learning algorithms. It provides a simulated environment to test your reinforcement learning algorithms easily with Atari games like Cartpole, Lunar Landing, or Breakout etc.

You can find the OpenAI Documentation [here](https://gym.openai.com/docs)

## Installation OpenAI Gym

```
# First install some system packages, you can use *homebrew* on macOS
brew install cmake boost boost-python sdl2 swig wget

# First clone the repo, then install all environments
git clone https://github.com/openai/gym.git
cd gym
pip install -e '.[all]'
```


## Using an environment

In [ ]:
import gym
env = gym.make('CartPole-v0')
env.reset()
env.render()

## Agent-Envrionment Loop

At each timestep, the agent chooses an action, and the environment returns an observation and a reward.

## Observations:

In [ ]:
The environment's step function returns four values:
 - observation (object): an environment-specific object representing your observation of the environment. 
        For example, pixel data from a camera, joint angles and joint velocities of a robot, or the board state in a board game.

 - reward (float): amount of reward achieved by the previous action. 
    The scale varies between environments, but the goal is always to increase your total reward.
    
 - done (boolean): whether it's time to reset the environment again. 
    Most (but not all) tasks are divided up into well-defined episodes, and done being True indicates the episode has terminated. 
    (For example, perhaps the pole tipped too far, or you lost your last life.)

 - info (dict): diagnostic information useful for debugging. 
    It can sometimes be useful for learning (for example, it might contain the raw probabilities behind the environment's last state change). However, official evaluations of your agent are not allowed to use this for learning.

## Cart-Pole: [Inverted pendulum problem](https://en.wikipedia.org/wiki/Inverted_pendulum)

In [ ]:
 - classic problem in dynamics and control theory 
 - widely used as a benchmark for testing control systems
 - exists analytical solutions based on Newton's Law 
 - we'll be using reinforcement learning

## Problem Formulation:

<img src="https://cdn-images-1.medium.com/max/1600/1*ohWngM-PVYmDG9KVpOm_xQ.gif", width=50%>
<center> Cart-Pole Env from OpenAI Gym </center>

In [ ]:
At each time step:
    
Observable States:
    - position (cart)
    - velocity (cart)
    - angle (pole)
    - angular velocity (pole)
    
Possible Actions:
    - move left
    - move right
    
State Space ( 4 x 1 continuous)
Action Space ( 2 x 1 discrete)

Reward:
    - pole is still upright (somewhat)
    - cart is within the bounding box

Done (environment restarts once done == True): 
    - pole is over a threshold angle
    or
    - cart is too far off outside the bounding box

Solved:
    The problem is considered “solved’ when it stays upright for over 195 time steps, 100 times consecutively.

## Policy

In [ ]:
We can describe a **policy** by specifying the agent action, given a state.

For example, one naive policy would be to just move the cart randomly. 
This would not be the optimal policy we are searching for, though.

## Rewards Table

## Q-Table

In [ ]:
Q matrix:
    rows: different states
    columns: actions
    initialize to be zero
    
The transition rule of Q learning is a very simple formula:

Q(state, action) = R(state, action) + Gamma * Max[Q(next state, all actions)]

## Using OpenAI gym for Cart-Pole

In [ ]:
# Just take a random action -- move left or move right at random
import gym

env = gym.make('CartPole-v0')
env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)

    if done:
        env.reset()
        break

# Code snippets for training

In [ ]:
import gym
import numpy as np
import random
import math
from time import sleep


In [ ]:
## Initialize the "Cart-Pole" environment
env = gym.make('CartPole-v0')

## Defining the environment related constants

# Number of discrete states (bucket) per state dimension
NUM_BUCKETS = (1, 1, 6, 3)  # (x, x', theta, theta')
# Number of discrete actions
NUM_ACTIONS = env.action_space.n # (left, right)
# Bounds for each discrete state
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]
# Index of the action
ACTION_INDEX = len(NUM_BUCKETS)

## Creating a Q-Table for each state-action pair
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))

## Learning related constants
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1

## Defining the simulation related constants
NUM_EPISODES = 1000
MAX_T = 250
STREAK_TO_END = 120
SOLVED_T = 199
DEBUG_MODE = True


In [ ]:
def simulate():

    ## Instantiating the learning related parameters
    learning_rate = get_learning_rate(0)
    explore_rate = get_explore_rate(0)
    discount_factor = 0.99  # since the world is unchanging

    num_streaks = 0

    for episode in range(NUM_EPISODES):

        # Reset the environment
        obv = env.reset()

        # the initial state
        state_0 = state_to_bucket(obv)

        for t in range(MAX_T):
            env.render()

            # Select an action
            action = select_action(state_0, explore_rate)

            # Execute the action
            obv, reward, done, _ = env.step(action)

            # Observe the result
            state = state_to_bucket(obv)

            # Update the Q based on the result
            best_q = np.amax(q_table[state])
            q_table[state_0 + (action,)] += learning_rate*(reward + discount_factor*(best_q) - q_table[state_0 + (action,)])

            # Setting up for the next iteration
            state_0 = state

            # Print data
            if (DEBUG_MODE):
                print("\nEpisode = %d" % episode)
                print("t = %d" % t)
                print("Action: %d" % action)
                print("State: %s" % str(state))
                print("Reward: %f" % reward)
                print("Best Q: %f" % best_q)
                print("Explore rate: %f" % explore_rate)
                print("Learning rate: %f" % learning_rate)
                print("Streaks: %d" % num_streaks)

                print("")

            if done:
                print("Episode %d finished after %f time steps" % (episode, t))
                if (t >= SOLVED_T):
                    num_streaks += 1
                else:
                    num_streaks = 0
                break


        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > STREAK_TO_END:
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)


In [ ]:
def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = np.argmax(q_table[state])
    return action

In [ ]:
def get_explore_rate(t):
    return max(MIN_EXPLORE_RATE, min(1, 1.0 - math.log10((t+1)/25)))


In [ ]:
def get_learning_rate(t):
    return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))


In [ ]:
def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)


In [ ]:
# Run the simulation

In [ ]:
if __name__ == "__main__":
    simulate()